In [9]:
import numpy as np
import itertools

In [10]:
## STEP 1: Profit allocation. Based on the paper by Naranyanan and Nardelli (2020).

# Calculate marginal contributions for each prosumer
def calculate_marginal_contributions(El, Ep, Cg, Cr, delta):
    num_prosumers = len(El)
    sum_El = np.sum(El)
    sum_Ep = np.sum(Ep)
    MCs = np.zeros(num_prosumers)   # Marginal contributions
    
    for i in range(num_prosumers):
        if sum_El <= sum_Ep: # If excess electricity can be sold to the aggregator
            if Ep[i] >= El[i]:
                # Case 1: if the prosumer produces more (or equal amount of) energy than consumes
                MCs[i] = delta * (Ep[i] - El[i]) * Cr
            else:
                # Case 2: if the prosumer consumes more energy than produces
                MCs[i] = (Ep[i] - El[i]) * Cr

        else:  # If the microgrid is not self-sufficient (more energy needs to be bought from the grid)
            if Ep[i] >= El[i]:
                # Case 1: if the prosumer produces more (or equal amount of) energy than consumes
                MCs[i] = (Ep[i] - El[i]) * Cg - delta * (Ep[i] - El[i]) * Cr
            else:
                # Case 2: if the prosumer consumes more energy than produces
                MCs[i] = (Ep[i] - El[i]) * Cg

    # Normalize MCs to avoid negative values
    MCs -= np.min(MCs)
    
    return MCs


# Calculate profit allocations (phi_i,new) for each customer i
def calculate_profit_allocations(El, Ep, Cg, Cr, delta):
    MCs = calculate_marginal_contributions(El, Ep, Cg, Cr, delta)
    num_prosumers = len(El)
    
    # Sum of MCs for normalization
    sum_MC = np.sum(MCs)
    
    allocations = np.zeros(num_prosumers)
    
    for i in range(num_prosumers):
        if Ep[i] >= El[i]:  # If the player produces more energy than consumes
            allocations[i] = MCs[i] / sum_MC * (El[i] * Cg + delta * (Ep[i] - El[i]) * Cr)
        else:  # If the player consumes more energy than produces
            allocations[i] = MCs[i] / sum_MC * Ep[i] * Cg
            
    return allocations

In [11]:
## Example use when the microgrid is self-sufficient and exess energy is sold to aggregator

# Define electricity consumption (El) and production (Ep) for each prosumer
# Example values
El = np.array([150, 400, 300, 100])  # Total electricity consumption for customers
Ep = np.array([200, 300, 350, 150])  # Total electricity production for customers

# Define prices of buying and selling electricity
Cg = 0.12  # Cost of purchasing electricity from the grid (€)
Cr = 0.15  # Price at which electricity is sold to an aggregator (€)
delta = 0.5  # Proportion of excess electricity sold to aggregator

profit_allocations = calculate_profit_allocations(El, Ep, Cg, Cr, delta)
print("Profit Allocations (€):", profit_allocations)

Profit Allocations (€): [ 7.25  0.   13.25  5.25]


In [12]:
## Example use when the microgrid is not self-sufficient and surplus energy is bought from the grid

# Define electricity consumption (El) and production (Ep) for each prosumer
# Example values
El = np.array([300, 200, 400, 150])  # Total electricity consumption for customers
Ep = np.array([200, 300, 350, 150])  # Total electricity production for customers

# Define prices of buying and selling electricity
Cg = 0.12  # Cost of purchasing electricity from the grid (€)
Cr = 0.15  # Price at which electricity is sold to an aggregator (€)
delta = 0.5  # Proportion of excess electricity sold to aggregator

profit_allocations = calculate_profit_allocations(El, Ep, Cg, Cr, delta)
print("Profit Allocations (€):", profit_allocations)

Profit Allocations (€): [ 0.         15.06521739  7.30434783  6.26086957]


In [13]:
## STEP 2: Simple optimization of energy allocation using Shapley values

class MicrogridNode:
    def __init__(self, id, supply, demand):
        self.id = id
        self.supply = supply  # Energy supply of the node
        self.demand = demand  # Energy demand of the node
        self.utility = 0      # Utility of the node after allocation
    
    def __repr__(self):
        return f"Node {self.id} (Supply: {self.supply}, Demand: {self.demand}, Utility: {self.utility})"

def calculate_shapley_value_demand(nodes,  total_energy):
    num_nodes = len(nodes)
    shapley_values = [0] * num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_demand = sum(node.demand for node in coalition)
            
            if coalition_demand <= total_energy:
                # Calculate marginal contribution of the i-th node of the permutation to the coalition
                if i == 0:
                    marginal_contribution = coalition_demand
                else:
                    prev_coalition_demand = sum(node.demand for node in perm[:i])
                    marginal_contribution = coalition_demand - prev_coalition_demand
                
                # Calculate Shapley value for i-th node of the permutation based on demand
                shapley_values[perm[i].id] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    
    return normalized_shapley_values

def allocate_energy(nodes, total_energy):
    shapley_values = calculate_shapley_value_demand(nodes, total_energy)
    
    # Allocate energy based on normalized Shapley values
    for i, node in enumerate(nodes):
        node.utility = shapley_values[i]
    
    return nodes

In [14]:
# Example usage
def main():
    # Define microgrid nodes (id, supply, demand)
    nodes = [
        MicrogridNode(0, 20, 6),
        MicrogridNode(1, 15, 12),
        MicrogridNode(2, 25, 15),
        MicrogridNode(3, 18, 15)
    ]
    
    total_available_energy = 78  # Total energy packets available (Sum of supply of microgrid nodes)
    
    # Allocate packets using the Shapley value approach based on demand
    allocated_nodes = allocate_energy(nodes, total_available_energy)
    
    # Display allocated nodes with updated utility values
    print("Allocated Nodes (Based on Demand):")
    for node in allocated_nodes:
        print(node)

    # Calculate and display the total allocated energy (sum of utilities)
    total_allocated_energy = sum(node.utility for node in allocated_nodes)
    print(f"Total Allocated Energy: {total_allocated_energy}")

main()

Allocated Nodes (Based on Demand):
Node 0 (Supply: 20, Demand: 6, Utility: 9.75)
Node 1 (Supply: 15, Demand: 12, Utility: 19.5)
Node 2 (Supply: 25, Demand: 15, Utility: 24.375)
Node 3 (Supply: 18, Demand: 15, Utility: 24.375)
Total Allocated Energy: 78.0


In [15]:
# STEP 3: Refined code for energy allocation.
# Each household uses their own energy first. Surplus energy is allocated between
# all of the households that have an energy deficit, using Shapley values for fair distribution.

class MicrogridNode:
    def __init__(self, id, supply, demand):
        self.id = id
        self.supply = supply  # Energy supply of the node
        self.demand = demand  # Energy demand of the node
        self.utility = 0      # Utility of the node after allocation
        self.balance = 0      # Energy balance (surplus or deficit)
    
    def __repr__(self):
        return f"Node {self.id} (Supply: {self.supply}, Demand: {self.demand}, Utility: {self.utility}, Balance: {self.balance})"

def calculate_shapley_value_buy(nodes, total_num_nodes, total_energy):
    ''' 
    Calculate the Shapley values for the deficit nodes. The calculation is based on how much
    deficit each node has. It will be used to determine how much energy each of them can buy.
    '''

    num_nodes = len(nodes)
    shapley_values = [0] * total_num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    if not permutations:
        return shapley_values  # No valid permutations, return zero Shapley values
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_demand = sum(node.supply - node.demand for node in coalition)
            
            if coalition_demand <= total_energy:
                # Calculate marginal contribution of the i-th node of the permutation to the coalition
                if i == 0:
                    marginal_contribution = coalition_demand
                else:
                    prev_coalition_demand = sum(node.supply - node.demand for node in perm[:i])
                    marginal_contribution = coalition_demand - prev_coalition_demand
            
                # Calculate Shapley value for the i-th node of the permutation based on demand
                shapley_values[perm[i].id] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    if shapley_sum != 0:
        normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    else:
        normalized_shapley_values = [0] * num_nodes
    
    return normalized_shapley_values

def calculate_shapley_value_sell(nodes, total_num_nodes, total_energy):
    '''
    Calculate the Shapley values for the surplus nodes. The calculation is based on how much
    surplus each of them has. It is used to determine how much energy each of them will sell.
    '''

    num_nodes = len(nodes)
    shapley_values = [0] * total_num_nodes
    
    # Calculate all permutations of nodes
    permutations = list(itertools.permutations(nodes))
    
    if not permutations:
        return shapley_values  # No valid permutations. The code should not reach here unless there are 0 nodes.
    
    for perm in permutations:
        for i in range(num_nodes):
            coalition = perm[:i+1]
            coalition_balance = sum(node.balance for node in coalition)
            
            # Calculate marginal contribution of the i-th node of the permutation to the coalition
            if i == 0:
                marginal_contribution = abs(coalition_balance)
            else:
                prev_coalition_balance = sum(node.balance for node in perm[:i])
                marginal_contribution = abs(coalition_balance - prev_coalition_balance)
            
            # Calculate Shapley value for the i-th node of the permutation based on balance
            shapley_values[perm[i].id] += marginal_contribution / num_nodes
    
    # Normalize Shapley values to ensure they sum up to total_energy
    shapley_sum = sum(shapley_values)
    if shapley_sum != 0:
        normalized_shapley_values = [sv * total_energy / shapley_sum for sv in shapley_values]
    else:
        normalized_shapley_values = [0] * num_nodes
    
    return normalized_shapley_values


def allocate_energy_refined(nodes):
    for node in nodes:
        node.balance = node.supply - node.demand

    # Divide microgrid nodes into surplus nodes (produce more energy than consume) and
    # deficit nodes (consume more energy than produce, so they need to buy some energy).
    # It is also possible that a node does not belong to either group (if it exactly
    # fills its own energy demand).
    surplus_nodes = [node for node in nodes if node.balance > 0]
    sum_surplus = 0
    for surplus_node in surplus_nodes:
        sum_surplus += surplus_node.balance

    deficit_nodes = [node for node in nodes if node.balance < 0]
    sum_deficit = 0
    for deficit_node in deficit_nodes:
        sum_deficit -= deficit_node.balance

    # The volume of trade inside the microgrid is the sum of deficit on the deficit nodes.
    # If less energy has been generated, it is the sum of surplus on the surplus nodes.
    trade_volume = min(sum_surplus, sum_deficit)

    if trade_volume <= 0:
        return nodes
    
    shapley_values_buy = calculate_shapley_value_buy(deficit_nodes, len(nodes), trade_volume)
    shapley_values_sell = calculate_shapley_value_sell(surplus_nodes, len(nodes), trade_volume)

    
    # Update utility based on final energy balance after trading
    for node in deficit_nodes:
        node.utility = node.supply + shapley_values_buy[node.id]
        node.balance = node.utility - node.demand

    for node in surplus_nodes:
        node.utility = node.supply - shapley_values_sell[node.id]
        node.balance = node.utility - node.demand
    
    return nodes



In [16]:
# Example usage
def main():
    # Define microgrid nodes (id, supply, demand)
    nodes = [
        MicrogridNode(0, 18, 5),
        MicrogridNode(1, 25, 6),
        MicrogridNode(2, 15, 17),
        MicrogridNode(3, 26, 7),
        MicrogridNode(4, 24, 29),
        MicrogridNode(5, 25, 29),
    ]
    
    
    # Allocate energy packets and simulate energy trading based on surplus Shapley values
    allocated_nodes = allocate_energy_refined(nodes)
    
    # Display allocated nodes with updated utility values and balance
    print("Allocated Nodes (Based on Surplus/Deficit):")
    for node in allocated_nodes:
        print(node)

    # Calculate and display the total allocated energy (sum of utilities)
    total_allocated_energy = sum(node.utility for node in allocated_nodes)
    print(f"Total Allocated Energy: {total_allocated_energy}")

main()

Allocated Nodes (Based on Surplus/Deficit):
Node 0 (Supply: 18, Demand: 5, Utility: 15.19607843137255, Balance: 10.19607843137255)
Node 1 (Supply: 25, Demand: 6, Utility: 20.901960784313726, Balance: 14.901960784313726)
Node 2 (Supply: 15, Demand: 17, Utility: 17.0, Balance: 0.0)
Node 3 (Supply: 26, Demand: 7, Utility: 21.901960784313726, Balance: 14.901960784313726)
Node 4 (Supply: 24, Demand: 29, Utility: 29.0, Balance: 0.0)
Node 5 (Supply: 25, Demand: 29, Utility: 29.0, Balance: 0.0)
Total Allocated Energy: 133.0
